# Initial imports cell

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import plotly.graph_objects as go

# Load and prepare data

In [59]:
df = pd.read_csv("AAPL.csv")
df['Date'] = pd.to_datetime(df['Date'])

# Calculate technical indicators

In [60]:
def calculate_signals(data, window=20):
    df = data.copy()
    # Moving averages
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['STD'] = df['Close'].rolling(window=window).std()
    # Upper and lower Bollinger Bands
    df['Upper_Band'] = df['SMA'] + (df['STD'] * 2)
    df['Lower_Band'] = df['SMA'] - (df['STD'] * 2)
    # Stop loss (2% below purchase price)
    df['Stop_Loss'] = df['Close'] * 0.98

    return df


# Prepare data for LSTM

In [61]:
def prepare_data(data, look_back=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    X, y = [], []
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i-look_back:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    # Use 80% for training
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    return X_train, X_test, y_train, y_test, scaler

# Build and train LSTM model

In [62]:
def build_model(look_back):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(look_back, 1)),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Generate trading signals

In [63]:
def generate_signals(actual, predicted, upper_band, lower_band):
    signals = pd.DataFrame(index=actual.index)
    signals['Price'] = actual
    signals['Predicted'] = predicted
    signals['Upper_Band'] = upper_band
    signals['Lower_Band'] = lower_band
    # Buy signal: Price crosses below lower band and predicted price is higher
    signals['Buy'] = (signals['Price'] < signals['Lower_Band']) & \
                    (signals['Predicted'] > signals['Price'])
    # Sell signal: Price crosses above upper band or predicted price is lower
    signals['Sell'] = (signals['Price'] > signals['Upper_Band']) | \
                     (signals['Predicted'] < signals['Price'] * 0.98)  # 2% stop loss

    return signals

# Main execution

In [64]:
look_back = 60
df_signals = calculate_signals(df)
X_train, X_test, y_train, y_test, scaler = prepare_data(df, look_back)

# Train model

In [65]:
model = build_model(look_back)
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




# Make predictions

In [66]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step


# Inverse transform predictions

In [67]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Combine predictions

In [68]:
predictions = np.concatenate([train_predict, test_predict])
df_predict = pd.DataFrame(predictions, index=df.index[look_back:], columns=['Predicted'])

# Generate trading signals

In [69]:
signals = generate_signals(
    df_signals['Close'][look_back:],
    df_predict['Predicted'],
    df_signals['Upper_Band'][look_back:],
    df_signals['Lower_Band'][look_back:]
)

# Create interactive plot with Plotly

In [70]:
fig = go.Figure()

# Add actual price

In [71]:
fig.add_trace(go.Scatter(
    x=signals.index,
    y=signals['Price'],
    name='Actual Price',
    line=dict(color='blue')
))

# Add predicted price

In [72]:
fig.add_trace(go.Scatter(
    x=signals.index,
    y=signals['Predicted'],
    name='Predicted Price',
    line=dict(color='orange', dash='dash')
))

# Add Bollinger Bands

In [73]:
fig.add_trace(go.Scatter(
    x=signals.index,
    y=signals['Upper_Band'],
    name='Upper Band',
    line=dict(color='gray', dash='dash')
))

fig.add_trace(go.Scatter(
    x=signals.index,
    y=signals['Lower_Band'],
    name='Lower Band',
    line=dict(color='gray', dash='dash')
))

# Add buy signals

In [74]:
fig.add_trace(go.Scatter(
    x=signals[signals['Buy']].index,
    y=signals[signals['Buy']]['Price'],
    name='Buy Signal',
    mode='markers',
    marker=dict(
        color='green',
        size=10,
        symbol='triangle-up'
    )
))

# Add sell signals

In [75]:
fig.add_trace(go.Scatter(
    x=signals[signals['Sell']].index,
    y=signals[signals['Sell']]['Price'],
    name='Sell Signal',
    mode='markers',
    marker=dict(
        color='red',
        size=10,
        symbol='triangle-down'
    )
))

# Add stop loss line

In [76]:
fig.add_trace(go.Scatter(
    x=signals.index,
    y=signals['Price'] * 0.98,
    name='Stop Loss (2%)',
    line=dict(color='red', dash='dot')
))

# Update layout

In [77]:
fig.update_layout(
    title='Stock Price Prediction with Trading Signals',
    xaxis_title='Date',
    yaxis_title='Price',
    hovermode='x unified',
    showlegend=True
)
fig.show()


# Print summary

In [78]:
print("\nTrading Signal Summary:")
print(f"Number of Buy Signals: {signals['Buy'].sum()}")
print(f"Number of Sell Signals: {signals['Sell'].sum()}")
print("\nStop Loss: 2% below purchase price")


Trading Signal Summary:
Number of Buy Signals: 107
Number of Sell Signals: 985

Stop Loss: 2% below purchase price
